In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from algorithm import (
                        NoRegularizationTrainer,
                        GradientTrainer,
                        EGTrainer,
                        CSVDataLoader,
                        MultiTaskModel,
                        MultiTaskDataset,
                        mlt_train_test_split,
                        # true_values_from_data_loader,
                        unique_value_counts,
                        Cindex,
                        brier_score,
                        )
import easydict
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

# Preparing Data

In [2]:
data = pd.read_csv('Dataset/processdata.csv', encoding='latin-1')
date_columns = ['Date.of.Last.Contact', 'Date.of.Diagnostic']
data[date_columns] = data[date_columns].apply(pd.to_datetime, errors='coerce')
data['Survival_Time'] = (data['Date.of.Last.Contact'] - data['Date.of.Diagnostic']).dt.days
data.loc[:, 'Survival_Time'] = data['Survival_Time'].replace({-1: 0})
data['indicater'] = np.where(data['Date.of.Death'].isna(), 0, 1)
# Staring time from 1
data['Survival_Time'] = data['Survival_Time'] + 1


columns_to_drop = ['Date.of.Death', 'Date.of.Last.Contact', 'Date.of.Diagnostic',
                   'Type.of.Death',  # High Correlation with 'Date.of.Death' and events
                   ]
data.drop(columns=columns_to_drop, inplace=True)
# print(data.isna().sum())
# data.dropna(inplace=True)

train_indices, test_indices = train_test_split(
        range(len(data)), stratify=data['indicater'], random_state=1, test_size=0.25
    )

X = data.drop(['Survival_Time', 'indicater'], axis=1)
time_all = data['Survival_Time'].values
event_all = data['indicater'].values

X_train = X.iloc[train_indices].copy()
X_test = X.iloc[test_indices].copy()
time_train = time_all[train_indices].copy()
time_test = time_all[test_indices].copy()
event_train = event_all[train_indices].copy()
event_test = event_all[test_indices].copy()


columns_to_one_hot = ['RCBP.Name', 'Raca.Color', 'State.Civil', 'Code.Profession', 'Name.Occupation',
                              'Status.Address',
                              'City.Address', 'Description.of.Topography', 'Topography.Code', 'Morphology.Description',
                              'Code.of.Morphology', 'Description.of.Disease', 'Illness.Code', 'Diagnostic.means',
                              'Extension',
                              #'Type.of.Death' # High Correlation with 'Date.of.Death' and events
                      ]

for column in columns_to_one_hot:
    top_9_values = X_train[column].value_counts().nlargest(9).index
    X_train[column] = X_train[column].where(X_train[column].isin(top_9_values), 'other')
    X_test[column] = X_test[column].where(X_test[column].isin(top_9_values), 'other')


X_train = pd.get_dummies(X_train, columns=columns_to_one_hot)
X_test = pd.get_dummies(X_test, columns=columns_to_one_hot)

print((X_test.columns == X_train.columns).all())

columns_to_binarize = ['Gender', 'Indicator.of.Rare.Case']    
for column in columns_to_binarize:
    lb = LabelBinarizer()
    X_train[column] = lb.fit_transform(X_train[column])
    X_test[column] = lb.transform(X_test[column])

scaler = MinMaxScaler()
X_train['Age'] = scaler.fit_transform(X_train[['Age']])
X_test['Age'] = scaler.transform(X_test[['Age']])

Tmax = time_train.max()
num_intervals=7
intervals = [(i * (Tmax // num_intervals), (i + 1) * (Tmax // num_intervals)) for i in range(num_intervals)]

Y_train = np.zeros((len(time_train), num_intervals), dtype=np.int_)
# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_train):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y_train[i, j] = 1
Y_train = torch.Tensor(Y_train)


Y_test = np.zeros((len(time_test), num_intervals), dtype=np.int_)
# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_test):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y_test[i, j] = 1
Y_test = torch.Tensor(Y_test)



# Creat mask matrix
W_train = np.zeros((len(time_train), num_intervals), dtype=np.int_)
# Until the time we know he was alive the value is 1, after that is 0
for i, (time_val, event_val) in enumerate(zip(time_train, event_train)):
    for j, (left, right) in enumerate(intervals):
        if event_val == 0 and time_val < left:
            W_train[i, j] = 0
        else:
            W_train[i, j] = 1
W_train = torch.Tensor(W_train)



# Creat mask matrix
W_test = np.zeros((len(time_test), num_intervals), dtype=np.int_)
# Until the time we know he was alive the value is 1, after that is 0
for i, (time_val, event_val) in enumerate(zip(time_test, event_test)):
    for j, (left, right) in enumerate(intervals):
        if event_val == 0 and time_val < left:
            W_test[i, j] = 0
        else:
            W_test[i, j] = 1
W_test = torch.Tensor(W_test)


for col in X_train.columns:
    if X_train[col].dtype == bool:
        X_train[col] = X_train[col].astype(int)
X_train = torch.tensor(X_train.values, dtype=torch.float32)

for col in X_test.columns:
    if X_test[col].dtype == bool:
        X_test[col] = X_test[col].astype(int)
X_test = torch.tensor(X_test.values, dtype=torch.float32)

# Y_train = Y[train_indices]
# Y_test = Y[test_indices]
# W_train = W[train_indices]
# W_test  = W[test_indices]
Y_train_transform = [Y_train[:, i:i + 1] for i in range(Y_train.size(1))]
Y_test_transform = [Y_test[:, i:i + 1] for i in range(Y_test.size(1))]
W_train_transform = [W_train[:, i:i+1] for i in range(W_train.size(1))]
W_test_transform = [W_test[:, i:i+1] for i in range(W_test.size(1))]

print((W_test_transform[-1] == 0).any())

args = easydict.EasyDict({
    "batch_size": 64,
    "lr": 0.01,
    "epochs": 200,
    "clip": 5.0,
    "lambda_reg": 0.01,
    "save_path": "outputfiles",
    "eg_k" : 1, 
    "early_stop_patience":15,
})

train_dataset = MultiTaskDataset(X_train, Y_train_transform, W_train_transform, event_train)
test_dataset = MultiTaskDataset(X_test, Y_test_transform, W_test_transform, event_test)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, drop_last=True)


True
tensor(True)


In [3]:
class EditedCindexOptimized(torch.nn.Module):

    def __init__(self):

        super(EditedCindexOptimized, self).__init__()

    def forward(self, y, y_hat, status):
        if not torch.is_tensor(y):
            y = torch.tensor(y, dtype=torch.float32)
        if not torch.is_tensor(y_hat):
            y_hat = torch.tensor(y_hat, dtype=torch.float32)
        if not torch.is_tensor(status):
            status = torch.tensor(status, dtype=torch.float32)

        # replacing loop acceleration with matrix calculation
        
        y_diff = y.unsqueeze(1) - y.unsqueeze(0)
        y_hat_diff = y_hat.unsqueeze(1) - y_hat.unsqueeze(0)
        # status[i] and status[j] mark whether to censored data
        status_i = status.unsqueeze(1)
        status_j = status.unsqueeze(0)
        valid_pairs = torch.logical_or((y_diff < 0) & (status_i == 1), (y_diff > 0) & (status_j == 1)).float()
        torch.diagonal(valid_pairs).fill_(0) #Diagonal set to 0 to eliminate interference
        concordant_pairs = torch.logical_or((y_diff < 0) & (y_hat_diff < 0)&(status_i == 1),(y_diff > 0) & (y_hat_diff > 0)& (status_j == 1)).float()
        torch.diagonal(concordant_pairs).fill_(0) #Diagonal set to 0 to eliminate interference
        concordant_pairs = concordant_pairs.float()
        c_index = concordant_pairs.sum() / valid_pairs.sum()
        return c_index.item()

# Results for Model with No regularization

In [4]:

model = MultiTaskModel(X_train.shape[1], Y_train.shape[1])
trainer = NoRegularizationTrainer(model,train_loader,test_loader, args)
trainer.train()


  0%|          | 1/200 [00:02<08:01,  2.42s/it]

End of Epoch 0, Average Training Loss: 1.8900, Average Gradient Norm: 0.0000
End of Epoch 0, Average Validation Loss: 1.8323
Current Learning Rate: 0.010000


  1%|          | 2/200 [00:03<05:03,  1.53s/it]

End of Epoch 1, Average Training Loss: 1.7339, Average Gradient Norm: 0.0000
End of Epoch 1, Average Validation Loss: 1.6235
Current Learning Rate: 0.010000


  2%|▏         | 3/200 [00:04<04:08,  1.26s/it]

...

End of Epoch 27, Average Training Loss: 1.4437, Average Gradient Norm: 0.0000
End of Epoch 27, Average Validation Loss: 1.6075
Current Learning Rate: 0.000010


 14%|█▍        | 29/200 [00:30<02:51,  1.00s/it]

End of Epoch 28, Average Training Loss: 1.4504, Average Gradient Norm: 0.0000
End of Epoch 28, Average Validation Loss: 1.6086
Current Learning Rate: 0.000010


 14%|█▍        | 29/200 [00:31<03:07,  1.09s/it]

End of Epoch 29, Average Training Loss: 1.4556, Average Gradient Norm: 0.0000
End of Epoch 29, Average Validation Loss: 1.6088
Current Learning Rate: 0.000010
Early stopping triggered after 30 epochs.


In [5]:



cindex_calculator_optimized = EditedCindexOptimized()

trainer.load_best_checkpoint()
predictions, Y_hat, Y_true, events = trainer.predict(train_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_train = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Training Data: {c11_train:.4f}")


predictions, Y_hat, Y_true, events = trainer.predict(test_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_test = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Test Data: {c11_test:.4f}")

Loaded model from checkpoint at epoch 15 with best validation loss 1.5687
C-index for Training Data: 0.7683
C-index for Test Data: 0.7536


# Results for EG Based Regularization Model

In [8]:
model = MultiTaskModel(X_train.shape[1], Y_train.shape[1])
trainer = EGTrainer(model,train_loader,test_loader, train_dataset, args)
trainer.train()


  0%|          | 1/200 [00:03<11:19,  3.42s/it]

End of Epoch 0, Average Training Loss: 2.3730, Average Gradient Norm: 2.8995
End of Epoch 0, Average Validation Loss: 1.7843
Current Learning Rate: 0.010000


  1%|          | 2/200 [00:06<11:02,  3.35s/it]

End of Epoch 1, Average Training Loss: 2.2220, Average Gradient Norm: 2.9356
End of Epoch 1, Average Validation Loss: 1.8864
Current Learning Rate: 0.010000


  2%|▏         | 3/200 [00:09<10:43,  3.27s/it]

...

End of Epoch 30, Average Training Loss: 2.0735, Average Gradient Norm: 4.7671
End of Epoch 30, Average Validation Loss: 1.7888
Current Learning Rate: 0.000010


 16%|█▌        | 32/200 [01:43<09:00,  3.22s/it]

End of Epoch 31, Average Training Loss: 2.0762, Average Gradient Norm: 4.7753
End of Epoch 31, Average Validation Loss: 1.7849
Current Learning Rate: 0.000010


 16%|█▌        | 32/200 [01:46<09:21,  3.34s/it]

End of Epoch 32, Average Training Loss: 2.0583, Average Gradient Norm: 4.5750
End of Epoch 32, Average Validation Loss: 1.7792
Current Learning Rate: 0.000010
Early stopping triggered after 33 epochs.


In [7]:
trainer.load_best_checkpoint()
predictions, Y_hat, Y_true, events = trainer.predict(train_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_train = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Training Data: {c11_train:.4f}")


predictions, Y_hat, Y_true, events = trainer.predict(test_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_test = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Test Data: {c11_test:.4f}")

Loaded model from checkpoint at epoch 2 with best validation loss 1.7398
C-index for Training Data: 0.6321
C-index for Test Data: 0.6308
